<a href="https://colab.research.google.com/github/MadmanMarble/Planet_U-NET/blob/main/Planet_training_data_organization_032024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 32.5 MB/s eta 0:00:00


In [3]:
import rasterio
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [4]:
!mkdir /content/labels

# !mkdir /content/OneYear
# !mkdir /content/OneYear/images


# !mkdir /content/TwoYears
# !mkdir /content/TwoYears/images

# !mkdir /content/ThreeYears
# !mkdir /content/ThreeYears/images


# Labels

In [5]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_modified_nwi_labels.zip -d /content/labels/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_modified_nwi_labels.zip
   creating: /content/labels/content/exports/modified_labels/
  inflating: /content/labels/content/exports/modified_labels/92_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/80_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/242_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/809_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/281_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/173_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/196_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/683_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/323_modif

In [26]:
images_folder = '/content/labels/content/exports/modified_labels'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'There are: {num_tif_files} files')

There are: 908 files


In [29]:
# Do the number of files in the folders dont match. Look at the data across one, two, three stack folders (UTM5)

def find_matching_files(images_folder, labels_folder):
    # List all .tif files in both folders
    images_files = [f for f in os.listdir(images_folder) if f.endswith('.tif')]
    labels_files = [f for f in os.listdir(labels_folder) if f.endswith('.tif')]

    # Extract the common identifiers from the filenames
    images_ids = set(sorted([f.split('_planet_stack_utm4.tif')[0] for f in images_files]))
    labels_ids = set(sorted([f.split('_modified_nwi_labels.tif')[0] for f in labels_files]))

    # Find the intersection of identifiers, i.e., those that exist in both folders
    matching_ids = images_ids.intersection(labels_ids)

    # Construct the filenames based on matching identifiers
    matching_images = sorted([f"{id}" for id in matching_ids])
    matching_labels = sorted([f"{id}" for id in matching_ids])

    return matching_images, matching_labels

# Specify the paths to your folders
images_folder = "/content/drive/MyDrive/data/model_template/x_training/x_training_1year"
labels_folder = "/content/labels/content/exports/modified_labels"


# Find matching files
matching_images, matching_labels = find_matching_files(images_folder, labels_folder)

print(len(matching_images))
print(len(matching_labels))
# Print the count of matching files
print(f"Found {len(matching_images)} matching image(s) and {len(matching_labels)} matching image(s).")

# Example output: matching filenames
for img, lbl in zip(matching_images, matching_labels):
    print(f"Image: {img} <--> label: {lbl}")

705
705
Found 705 matching image(s) and 705 matching image(s).
Image: 1 <--> label: 1
Image: 10 <--> label: 10
Image: 100 <--> label: 100
Image: 101 <--> label: 101
Image: 102 <--> label: 102
Image: 103 <--> label: 103
Image: 104 <--> label: 104
Image: 105 <--> label: 105
Image: 106 <--> label: 106
Image: 107 <--> label: 107
Image: 109 <--> label: 109
Image: 11 <--> label: 11
Image: 110 <--> label: 110
Image: 111 <--> label: 111
Image: 112 <--> label: 112
Image: 113 <--> label: 113
Image: 114 <--> label: 114
Image: 115 <--> label: 115
Image: 116 <--> label: 116
Image: 117 <--> label: 117
Image: 118 <--> label: 118
Image: 119 <--> label: 119
Image: 12 <--> label: 12
Image: 120 <--> label: 120
Image: 121 <--> label: 121
Image: 122 <--> label: 122
Image: 124 <--> label: 124
Image: 125 <--> label: 125
Image: 126 <--> label: 126
Image: 127 <--> label: 127
Image: 128 <--> label: 128
Image: 129 <--> label: 129
Image: 13 <--> label: 13
Image: 130 <--> label: 130
Image: 131 <--> label: 131
Imag

In [32]:
import re

def extract_numeric_id(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else None

# Assuming you've defined matching_images and matching_labels
# images_files = [f for f in os.listdir(images_folder) if f.endswith('.tif')]
# labels_files = [f for f in os.listdir(labels_folder) if f.endswith('.tif')]


# image_ids = [extract_numeric_id(img) for img in os.listdir(images_folder) if extract_numeric_id.endswith('.tif')]
# label_ids = [extract_numeric_id(lbl) for lbl in os.listdir(labels_folder) if extract_numeric_id.endswith('.tif')]

image_ids = [extract_numeric_id(img) for img in matching_images]
label_ids = [extract_numeric_id(lbl) for lbl in matching_labels]

min_id = min(min(image_ids), min(label_ids))
max_id = max(max(image_ids), max(label_ids))

all_ids = set(range(min_id, max_id + 2))
missing_ids = all_ids - set(image_ids) - set(label_ids)

print("Missing IDs:", sorted(list(missing_ids)))

print(f'There are {len(missing_ids)} missing IDs.')


Missing IDs: [8, 16, 24, 32, 40, 41, 42, 43, 44, 45, 46, 47, 48, 63, 78, 93, 108, 123, 138, 153, 155, 168, 170, 198, 213, 221, 228, 243, 253, 258, 273, 281, 288, 303, 318, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 367, 386, 405, 424, 443, 462, 481, 500, 519, 538, 557, 576, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 644, 655, 666, 677, 688, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 762, 764, 765, 766, 767, 768, 782, 783, 784, 785, 786, 800, 801, 802, 803, 804, 818, 819, 820, 821, 822, 823, 828, 836, 837, 838, 839, 840, 854, 855, 856, 857, 858, 872, 873, 874, 875, 876, 890, 891, 892, 893

In [33]:
# prompt: print the numeber of missing ids

print(f'There are {len(missing_ids)} missing IDs.')


There are 203 missing IDs.


In [34]:
import os
import shutil
import re

# The directory containing the files you want to check
source_dir = "/content/labels/content/exports/modified_labels"

# The target directory for moving the matching files
target_dir = "/content/labels_removed"

# Ensure the target directory exists, create if it does not
os.makedirs(target_dir, exist_ok=True)

# Function to extract numeric ID from filename
def extract_numeric_id(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else None

# Your list of missing IDs (assuming you have this from the previous step)
missing_ids = {8, 16, 24, 32, 40, 41, 42, 43, 44, 45, 46, 47, 48, 63, 78, 93, 108, 123, 138, 153,
               155, 168, 170, 198, 213, 221, 228, 243, 253, 258, 273, 281, 288, 303, 318, 333, 334,
               335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 367, 386, 405,
               424, 443, 462, 481, 500, 519, 538, 557, 576, 595, 596, 597, 598, 599, 600, 601, 602,
               603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619,
               620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 644, 655, 666,
               677, 688, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713,
               714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730,
               731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747,
               748, 749, 750, 762, 764, 765, 766, 767, 768, 782, 783, 784, 785, 786, 800, 801, 802,
               803, 804, 818, 819, 820, 821, 822, 823, 828, 836, 837, 838, 839, 840, 854, 855, 856,
               857, 858, 872, 873, 874, 875, 876, 890, 891, 892, 893, 894, 908}

# Iterate over files in the source directory
for filename in os.listdir(source_dir):
    file_id = extract_numeric_id(filename)

    # Check if the current file's ID is in the list of missing IDs
    if file_id in missing_ids:
        # Construct the full source path
        source_path = os.path.join(source_dir, filename)

        # Construct the full target path
        target_path = os.path.join(target_dir, filename)

        # Move the file
        shutil.move(source_path, target_path)
        print(f"Moved: {filename}")


Moved: 281_modified_nwi_labels.tif
Moved: 741_modified_nwi_labels.tif
Moved: 630_modified_nwi_labels.tif
Moved: 626_modified_nwi_labels.tif
Moved: 334_modified_nwi_labels.tif
Moved: 620_modified_nwi_labels.tif
Moved: 622_modified_nwi_labels.tif
Moved: 890_modified_nwi_labels.tif
Moved: 729_modified_nwi_labels.tif
Moved: 273_modified_nwi_labels.tif
Moved: 596_modified_nwi_labels.tif
Moved: 621_modified_nwi_labels.tif
Moved: 786_modified_nwi_labels.tif
Moved: 801_modified_nwi_labels.tif
Moved: 608_modified_nwi_labels.tif
Moved: 253_modified_nwi_labels.tif
Moved: 836_modified_nwi_labels.tif
Moved: 828_modified_nwi_labels.tif
Moved: 823_modified_nwi_labels.tif
Moved: 804_modified_nwi_labels.tif
Moved: 557_modified_nwi_labels.tif
Moved: 820_modified_nwi_labels.tif
Moved: 40_modified_nwi_labels.tif
Moved: 609_modified_nwi_labels.tif
Moved: 346_modified_nwi_labels.tif
Moved: 746_modified_nwi_labels.tif
Moved: 734_modified_nwi_labels.tif
Moved: 606_modified_nwi_labels.tif
Moved: 45_modified_nw

In [36]:
images_folder = '/content/labels/content/exports/modified_labels'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'There are: {num_tif_files} files')

There are: 705 files


In [37]:
images_folder = '/content/labels_removed'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'There are: {num_tif_files} files')

There are: 203 files


In [38]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_modified_nwi_labels_02.zip -d /content/labels/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_modified_nwi_labels_02.zip
   creating: /content/labels/content/exports/modified_labels/
  inflating: /content/labels/content/exports/modified_labels/2067_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/1430_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/2168_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/2342_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/1820_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/2485_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/2351_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_labels/1494_modified_nwi_labels.tif  
  inflating: /content/labels/content/exports/modified_lab

In [39]:
images_folder = '/content/labels/content/exports/modified_labels'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'There are: {num_tif_files} files')

There are: 1740 files


In [40]:
def find_matching_files(images_folder, labels_folder):
    # List all .tif files in both folders
    images_files = [f for f in os.listdir(images_folder) if f.endswith('.tif')]
    labels_files = [f for f in os.listdir(labels_folder) if f.endswith('.tif')]

    # Extract the common identifiers from the filenames
    images_ids = set(sorted([f.split('_planet_stack_utm4.tif')[0] for f in images_files]))
    labels_ids = set(sorted([f.split('_modified_nwi_labels.tif')[0] for f in labels_files]))

    # Find the intersection of identifiers, i.e., those that exist in both folders
    matching_ids = images_ids.intersection(labels_ids)

    # Construct the filenames based on matching identifiers
    matching_images = sorted([f"{id}" for id in matching_ids])
    matching_labels = sorted([f"{id}" for id in matching_ids])

    return matching_images, matching_labels

# Specify the paths to your folders
images_folder = "/content/drive/MyDrive/data/model_template/x_training/x_training_1year"
labels_folder = "/content/labels/content/exports/modified_labels"


# Find matching files
matching_images, matching_labels = find_matching_files(images_folder, labels_folder)

print(len(matching_images))
print(len(matching_labels))
# Print the count of matching files
print(f"Found {len(matching_images)} matching image(s) and {len(matching_labels)} matching image(s).")

# Example output: matching filenames
for img, lbl in zip(matching_images, matching_labels):
    print(f"Image: {img} <--> label: {lbl}")

1480
1480
Found 1480 matching image(s) and 1480 matching image(s).
Image: 1007 <--> label: 1007
Image: 1008 <--> label: 1008
Image: 1009 <--> label: 1009
Image: 1010 <--> label: 1010
Image: 1011 <--> label: 1011
Image: 1012 <--> label: 1012
Image: 1013 <--> label: 1013
Image: 1014 <--> label: 1014
Image: 1015 <--> label: 1015
Image: 1016 <--> label: 1016
Image: 1017 <--> label: 1017
Image: 1018 <--> label: 1018
Image: 1019 <--> label: 1019
Image: 1020 <--> label: 1020
Image: 1021 <--> label: 1021
Image: 1022 <--> label: 1022
Image: 1023 <--> label: 1023
Image: 1024 <--> label: 1024
Image: 1025 <--> label: 1025
Image: 1026 <--> label: 1026
Image: 1027 <--> label: 1027
Image: 1028 <--> label: 1028
Image: 1029 <--> label: 1029
Image: 1030 <--> label: 1030
Image: 1031 <--> label: 1031
Image: 1033 <--> label: 1033
Image: 1034 <--> label: 1034
Image: 1035 <--> label: 1035
Image: 1036 <--> label: 1036
Image: 1037 <--> label: 1037
Image: 1038 <--> label: 1038
Image: 1039 <--> label: 1039
Image

In [46]:
def extract_numeric_id(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else None


image_ids = [extract_numeric_id(img) for img in matching_images]
label_ids = [extract_numeric_id(lbl) for lbl in matching_labels]

min_id = min(min(image_ids), min(label_ids))
max_id = max(max(image_ids), max(label_ids))

all_ids = set(range(min_id, max_id + 2))
missing_ids = all_ids - set(image_ids) - set(label_ids)

print("Missing IDs:", sorted(list(missing_ids)))

print(f'There are {len(missing_ids)} missing IDs.')


Missing IDs: [942, 958, 974, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1032, 1058, 1084, 1110, 1136, 1162, 1188, 1214, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1277, 1288, 1299, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1328, 1335, 1342, 1349, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1370, 1379, 1395, 1411, 1427, 1443, 1459, 1472, 1475, 1491, 1498, 1507, 1523, 1530, 1532, 1539, 1546, 1555, 1557, 1571, 1587, 1603, 1619, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1666, 1681, 1696, 1711, 1726, 1741, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1776, 1781, 1786, 1791, 1796, 1801, 1806, 1811, 1816, 1821, 1826, 1831, 1832, 1833, 1834, 1835, 1836, 1845, 1854, 1863, 1872, 1881, 189

In [47]:
# The directory containing the files you want to check
source_dir = "/content/labels/content/exports/modified_labels"

# The target directory for moving the matching files
target_dir = "/content/labels_removed"

# Ensure the target directory exists, create if it does not
os.makedirs(target_dir, exist_ok=True)

# Function to extract numeric ID from filename
def extract_numeric_id(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else None

# Your list of missing IDs (assuming you have this from the previous step)
missing_ids = {942, 958, 974, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1032, 1058, 1084,
               1110, 1136, 1162, 1188, 1214, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255,
               1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1277, 1288, 1299, 1310, 1311, 1312, 1313, 1314, 1315, 1316,
               1317, 1318, 1319, 1320, 1321, 1328, 1335, 1342, 1349, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1370, 1379, 1395, 1411,
               1427, 1443, 1459, 1472, 1475, 1491, 1498, 1507, 1523, 1530, 1532, 1539, 1546, 1555, 1557, 1571, 1587, 1603, 1619, 1635, 1636,
               1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1666, 1681, 1696, 1711, 1726, 1741,
               1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1776, 1781, 1786, 1791, 1796,
               1801, 1806, 1811, 1816, 1821, 1826, 1831, 1832, 1833, 1834, 1835, 1836, 1845, 1854, 1863, 1872, 1881, 1890, 1899, 1908, 1917,
               1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1957, 1988, 2019, 2050, 2081, 2112, 2118, 2143, 2174, 2179, 2205, 2236,
               2267, 2298, 2329, 2355, 2360, 2391, 2422, 2453, 2484, 2515, 2516, 2517, 2518, 2519, 2520, 2521, 2522, 2523, 2524, 2525, 2526,
               2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 2541, 2542, 2543, 2544, 2545, 2546, 2551,
               2556, 2561, 2566, 2571, 2572, 2573, 2574, 2575, 2576, 2586, 2596, 2606, 2616, 2626, 2636, 2646, 2656}

# Iterate over files in the source directory
for filename in os.listdir(source_dir):
    file_id = extract_numeric_id(filename)

    # Check if the current file's ID is in the list of missing IDs
    if file_id in missing_ids:
        # Construct the full source path
        source_path = os.path.join(source_dir, filename)

        # Construct the full target path
        target_path = os.path.join(target_dir, filename)

        # Move the file
        shutil.move(source_path, target_path)
        print(f"Moved: {filename}")


Moved: 1649_modified_nwi_labels.tif
Moved: 1261_modified_nwi_labels.tif
Moved: 1921_modified_nwi_labels.tif
Moved: 1248_modified_nwi_labels.tif
Moved: 1711_modified_nwi_labels.tif
Moved: 1498_modified_nwi_labels.tif
Moved: 1826_modified_nwi_labels.tif
Moved: 1259_modified_nwi_labels.tif
Moved: 1642_modified_nwi_labels.tif
Moved: 996_modified_nwi_labels.tif
Moved: 1638_modified_nwi_labels.tif
Moved: 1899_modified_nwi_labels.tif
Moved: 2636_modified_nwi_labels.tif
Moved: 1263_modified_nwi_labels.tif
Moved: 1816_modified_nwi_labels.tif
Moved: 2526_modified_nwi_labels.tif
Moved: 1768_modified_nwi_labels.tif
Moved: 1648_modified_nwi_labels.tif
Moved: 2532_modified_nwi_labels.tif
Moved: 1255_modified_nwi_labels.tif
Moved: 1359_modified_nwi_labels.tif
Moved: 1908_modified_nwi_labels.tif
Moved: 1328_modified_nwi_labels.tif
Moved: 990_modified_nwi_labels.tif
Moved: 992_modified_nwi_labels.tif
Moved: 2360_modified_nwi_labels.tif
Moved: 2422_modified_nwi_labels.tif
Moved: 1539_modified_nwi_labels

In [48]:
images_folder = '/content/labels/content/exports/modified_labels'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'There are: {num_tif_files} files')

There are: 1490 files


In [ ]:
import os
import re

# Define the paths to your folders
x_training_path = '/content/drive/MyDrive/data/model_template/x_training/x_training_1year'
y_training_path = '/content/labels/content/exports/modified_labels'

# List files in both folders
x_files = os.listdir(x_training_path)
y_files = os.listdir(y_training_path)

# Compile a regex pattern to extract the numeric identifier and utm code
pattern = re.compile('(\d+)_planet_stack_utm(\d+)\.tif')

# Initialize a list to keep track of missing matches
missing_matches = []

# Iterate over files in the x_training folder
for x_file in x_files:
    match = pattern.match(x_file)
    if match:
        # Construct the corresponding y_training file name
        # numeric_id, utm_code = match.groups()
        # y_file_expected = f"{numeric_id}_nwi_ccap_filled_utm{utm_code}.tif"
        numeric_id, utm_code = match.groups()
        y_file_expected = f"{numeric_id}_modified_nwi_labels.tif"

        # Check if this file exists in the y_training folder
        if y_file_expected not in y_files:
            # Append tuples of (numeric identifier as int, x_file, y_file_expected) to sort by identifier
            missing_matches.append((int(numeric_id), x_file, y_file_expected))

# Sort missing_matches by the numeric identifier
missing_matches.sort(key=lambda x: x[0])

# Output the results
if missing_matches:
    print("Missing matches found:")
    for _, x, y in missing_matches:
        print(f"{x} in x_training does not have a corresponding {y} in y_training.")
else:
    print("All files in x_training have corresponding matches in y_training.")

# OneYear UTM 4 Data

In [ ]:
!unzip /content/drive/MyDrive/data/model_template/x_training/x_training_1year/x_training_1year.zip -d /content/OneYear/images/

Archive:  /content/drive/MyDrive/data/model_template/x_training/x_training_1year/x_training_1year.zip
   creating: /content/OneYear/images/x_training_1year/
  inflating: /content/OneYear/images/x_training_1year/1007_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1008_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1009_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/100_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1010_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1011_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1012_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1013_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1014_planet_stack_utm4.tif  
  inflating: /content/OneYear/images/x_training_1year/1015_planet_stack_utm4.tif  
  inflating: /

In [ ]:
!unzip /content/drive/MyDrive/data/model_template/x_training/x_training_2year/x_training_2year.zip -d /content/TwoYears/images/

Archive:  /content/drive/MyDrive/data/model_template/x_training/x_training_2year/x_training_2year.zip
   creating: /content/TwoYears/images/x_training_2year/
  inflating: /content/TwoYears/images/x_training_2year/1007_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1008_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1009_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/100_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1010_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1011_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1012_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1013_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1014_planet_stack_utm4.tif  
  inflating: /content/TwoYears/images/x_training_2year/1015_planet_stack_utm4.tif  
  i

In [ ]:
!unzip /content/drive/MyDrive/data/model_template/x_training/x_training_3year/x_training_3year.zip -d /content/ThreeYears/images/

Archive:  /content/drive/MyDrive/data/model_template/x_training/x_training_3year/x_training_3year.zip
   creating: /content/ThreeYears/images/x_training_3year/
  inflating: /content/ThreeYears/images/x_training_3year/1007_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1008_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1009_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/100_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1010_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1011_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1012_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1013_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1014_planet_stack_utm4.tif  
  inflating: /content/ThreeYears/images/x_training_3year/1015_plane

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/images/x_training_1year'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'The number of .tif files in the folder is: {num_tif_files}')

images_folder = '/content/TwoYears/images/x_training_2year'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'The number of .tif files in the folder is: {num_tif_files}')


images_folder = '/content/ThreeYears/images/x_training_3year'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'The number of .tif files in the folder is: {num_tif_files}')


The number of .tif files in the folder is: 2392
The number of .tif files in the folder is: 2392
The number of .tif files in the folder is: 2392


In [ ]:
# Do the number of files in the folders dont match. Look at the data across one, two, three stack folders (UTM5)

def find_matching_files(images_folder, labels_folder):
    # List all .tif files in both folders
    images_files = [f for f in os.listdir(images_folder) if f.endswith('.tif')]
    labels_files = [f for f in os.listdir(labels_folder) if f.endswith('.tif')]

    # Extract the common identifiers from the filenames
    images_ids = set(sorted([f.split('_planet_stack.tif')[0] for f in images_files]))
    labels_ids = set(sorted([f.split('_planet_stack.tif')[0] for f in labels_files]))

    # Find the intersection of identifiers, i.e., those that exist in both folders
    matching_ids = images_ids.intersection(labels_ids)

    # Construct the filenames based on matching identifiers
    matching_images = [f"{id}" for id in matching_ids]
    matching_labels = [f"{id}" for id in matching_ids]

    return matching_images, matching_labels

# Specify the paths to your folders
images_folder = "/content/TwoYears/images/x_training_2year"
labels_folder = "/content/ThreeYears/images/x_training_3year"

# Find matching files
matching_images, matching_labels = find_matching_files(images_folder, labels_folder)

# Print the count of matching files
print(f"Found {len(matching_images)} matching image(s) and {len(matching_labels)} matching image(s).")

# Example output: matching filenames
for img, lbl in zip(matching_images, matching_labels):
    print(f"Image: {img} <--> Image: {lbl}")

Found 2392 matching image(s) and 2392 matching image(s).
Image: 1440_planet_stack_utm4.tif <--> Image: 1440_planet_stack_utm4.tif
Image: 130_planet_stack_utm4.tif <--> Image: 130_planet_stack_utm4.tif
Image: 2068_planet_stack_utm4.tif <--> Image: 2068_planet_stack_utm4.tif
Image: 608_planet_stack_utm4.tif <--> Image: 608_planet_stack_utm4.tif
Image: 1616_planet_stack_utm4.tif <--> Image: 1616_planet_stack_utm4.tif
Image: 1665_planet_stack_utm4.tif <--> Image: 1665_planet_stack_utm4.tif
Image: 1215_planet_stack_utm4.tif <--> Image: 1215_planet_stack_utm4.tif
Image: 2256_planet_stack_utm4.tif <--> Image: 2256_planet_stack_utm4.tif
Image: 2089_planet_stack_utm4.tif <--> Image: 2089_planet_stack_utm4.tif
Image: 2604_planet_stack_utm4.tif <--> Image: 2604_planet_stack_utm4.tif
Image: 520_planet_stack_utm4.tif <--> Image: 520_planet_stack_utm4.tif
Image: 748_planet_stack_utm4.tif <--> Image: 748_planet_stack_utm4.tif
Image: 1206_planet_stack_utm4.tif <--> Image: 1206_planet_stack_utm4.tif
Im

In [ ]:
# Specify the folder path
folder_path = '/content/OneYear/images/x_training_1year'

# Function to extract the numeric part of the filename
def extract_numeric_part(filename):
    match = re.match(r"(\d+)", filename)
    return int(match.group()) if match else None

# List all files in the folder
files = os.listdir(folder_path)

# Extract numeric parts and sort
numeric_parts = [extract_numeric_part(file) for file in files if extract_numeric_part(file) is not None]
sorted_numeric_parts = sorted(numeric_parts)

# Find missing numbers
max_num = sorted_numeric_parts[-1]  # Assuming the largest number in the sequence is the last one
complete_range = set(range(1, max_num + 1))  # Start from 1 to max_num, inclusive
missing_numbers = complete_range - set(sorted_numeric_parts)

# Print missing numbers
print("Missing numbers:", sorted(list(missing_numbers)))

Missing numbers: [8, 32, 40, 41, 45, 46, 78, 123, 198, 213, 221, 228, 243, 253, 258, 273, 281, 288, 303, 318, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 367, 386, 405, 424, 443, 462, 481, 500, 519, 538, 557, 576, 595, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 644, 655, 666, 677, 688, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 728, 746, 762, 764, 782, 800, 803, 818, 823, 828, 836, 854, 872, 890, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 942, 958, 974, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1032, 1058, 1084, 1110, 1136, 1162, 1188, 1214, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1277, 1288, 1299, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/labels'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2374


In [ ]:
import os
import re

# Define the paths to your folders
x_training_path = "/content/TwoYears/images/x_training_2year"
y_training_path = "/content/labels"

# List files in both folders
x_files = os.listdir(x_training_path)
y_files = os.listdir(y_training_path)

# Compile a regex pattern to extract the numeric identifier and utm code
pattern = re.compile('(\d+)_planet_stack_utm(\d+)\.tif')

# Initialize a list to keep track of missing matches
missing_matches = []

# Iterate over files in the x_training folder
for x_file in x_files:
    match = pattern.match(x_file)
    if match:
        # Construct the corresponding y_training file name
        numeric_id, utm_code = match.groups()
        y_file_expected = f"{numeric_id}_nwi_ccap_filled_utm{utm_code}.tif"

        # Check if this file exists in the y_training folder
        if y_file_expected not in y_files:
            missing_matches.append((x_file, y_file_expected))

# Output the results
if missing_matches:
    print("Missing matches found:")
    for x, y in missing_matches:
        print(f"{x} in x_training does not have a corresponding {y} in y_training.")
else:
    print("All files in x_training have corresponding matches in y_training.")


All files in x_training have corresponding matches in y_training.


In [ ]:
import os
import re

# Define the paths to your folders
x_training_path = '/content/OneYear/images/x_training_1year'
y_training_path = '/content/labels'

# List files in both folders
x_files = os.listdir(x_training_path)
y_files = os.listdir(y_training_path)

# Compile a regex pattern to extract the numeric identifier and utm code
pattern = re.compile('(\d+)_planet_stack_utm(\d+)\.tif')

# Initialize a list to keep track of missing matches
missing_matches = []

# Iterate over files in the x_training folder
for x_file in x_files:
    match = pattern.match(x_file)
    if match:
        # Construct the corresponding y_training file name
        numeric_id, utm_code = match.groups()
        y_file_expected = f"{numeric_id}_nwi_ccap_filled_utm{utm_code}.tif"

        # Check if this file exists in the y_training folder
        if y_file_expected not in y_files:
            # Append tuples of (numeric identifier as int, x_file, y_file_expected) to sort by identifier
            missing_matches.append((int(numeric_id), x_file, y_file_expected))

# Sort missing_matches by the numeric identifier
missing_matches.sort(key=lambda x: x[0])

# Output the results
if missing_matches:
    print("Missing matches found:")
    for _, x, y in missing_matches:
        print(f"{x} in x_training does not have a corresponding {y} in y_training.")
else:
    print("All files in x_training have corresponding matches in y_training.")


Missing matches found:
596_planet_stack_utm4.tif in x_training does not have a corresponding 596_nwi_ccap_filled_utm4.tif in y_training.
597_planet_stack_utm4.tif in x_training does not have a corresponding 597_nwi_ccap_filled_utm4.tif in y_training.
598_planet_stack_utm4.tif in x_training does not have a corresponding 598_nwi_ccap_filled_utm4.tif in y_training.
599_planet_stack_utm4.tif in x_training does not have a corresponding 599_nwi_ccap_filled_utm4.tif in y_training.
600_planet_stack_utm4.tif in x_training does not have a corresponding 600_nwi_ccap_filled_utm4.tif in y_training.
601_planet_stack_utm4.tif in x_training does not have a corresponding 601_nwi_ccap_filled_utm4.tif in y_training.
602_planet_stack_utm4.tif in x_training does not have a corresponding 602_nwi_ccap_filled_utm4.tif in y_training.
603_planet_stack_utm4.tif in x_training does not have a corresponding 603_nwi_ccap_filled_utm4.tif in y_training.
604_planet_stack_utm4.tif in x_training does not have a correspon

In [ ]:
!mkdir /content/moved_x_training_1year
!mkdir /content/moved_x_training_2year
!mkdir /content/moved_x_training_3year

In [ ]:
import os
import re
import shutil

# To move files out of x training data folder bc they dont exist in y training data folder

# Define the paths to your folders
x_training_path = '/content/ThreeYears/images/x_training_3year'
y_training_path = '/content/labels'
missing_x_training_path = '/content/moved_x_training_3year'

# Ensure the target directory exists
if not os.path.exists(missing_x_training_path):
    os.makedirs(missing_x_training_path)

# List files in both folders
x_files = os.listdir(x_training_path)
y_files = os.listdir(y_training_path)

# Compile a regex pattern to extract the numeric identifier and utm code
pattern = re.compile('(\d+)_planet_stack_utm(\d+)\.tif')

# Initialize a list to keep track of missing matches
missing_matches = []

# Iterate over files in the x_training folder
for x_file in x_files:
    match = pattern.match(x_file)
    if match:
        numeric_id, utm_code = match.groups()
        y_file_expected = f"{numeric_id}_nwi_ccap_filled_utm{utm_code}.tif"
        if y_file_expected not in y_files:
            missing_matches.append((int(numeric_id), x_file, y_file_expected))
            # Construct the full path to the file to be moved
            file_to_move = os.path.join(x_training_path, x_file)
            # Construct the full target path
            target_path = os.path.join(missing_x_training_path, x_file)
            # Move the file
            shutil.move(file_to_move, target_path)

# Sort missing_matches by the numeric identifier for reporting
missing_matches.sort(key=lambda x: x[0])

# Output the results
if missing_matches:
    print("Missing matches found and moved:")
    for _, x, y in missing_matches:
        print(f"{x} in x_training does not have a corresponding {y} in y_training. Moved {x} to {missing_x_training_path}.")
else:
    print("All files in x_training have corresponding matches in y_training.")


All files in x_training have corresponding matches in y_training.


In [ ]:
!mkdir /content/moved_y_training

In [ ]:
import os
import re
import shutil

# To move files out of y training data folder bc they dont exist in x training data folder


# Define the paths to your folders
x_training_path = '/content/ThreeYears/images/x_training_3year'
y_training_path = '/content/labels'
missing_y_training_path = '/content/moved_y_training'  # Path where non-corresponding y_training files will be moved

# Ensure the missing_y_training_path exists
if not os.path.exists(missing_y_training_path):
    os.makedirs(missing_y_training_path)

# List files in both folders
x_files = os.listdir(x_training_path)
y_files = os.listdir(y_training_path)

# Pattern for extracting identifier and UTM from x_training files
x_pattern = re.compile('(\d+)_planet_stack_utm(\d+)\.tif')

# Pattern for extracting identifier and UTM from y_training files
y_pattern = re.compile('(\d+)_nwi_ccap_filled_utm(\d+)\.tif')

# Check each file in y_training against x_training
for y_file in y_files:
    match = y_pattern.match(y_file)
    if match:
        numeric_id, utm_code = match.groups()
        # Construct the expected corresponding x_training file name
        x_file_expected = f"{numeric_id}_planet_stack_utm{utm_code}.tif"

        # If the expected x_file does not exist in x_training, move y_file to missing_y_training_path
        if x_file_expected not in x_files:
            src_path = os.path.join(y_training_path, y_file)
            dest_path = os.path.join(missing_y_training_path, y_file)
            shutil.move(src_path, dest_path)
            print(f"Moved {y_file} to {missing_y_training_path} because it does not have a corresponding file in x_training.")


Moved 2019_nwi_ccap_filled_utm4.tif to /content/moved_y_training because it does not have a corresponding file in x_training.
Moved 677_nwi_ccap_filled_utm4.tif to /content/moved_y_training because it does not have a corresponding file in x_training.
Moved 205_nwi_ccap_filled_utm5.tif to /content/moved_y_training because it does not have a corresponding file in x_training.
Moved 84_nwi_ccap_filled_utm5.tif to /content/moved_y_training because it does not have a corresponding file in x_training.
Moved 1764_nwi_ccap_filled_utm4.tif to /content/moved_y_training because it does not have a corresponding file in x_training.
Moved 1004_nwi_ccap_filled_utm4.tif to /content/moved_y_training because it does not have a corresponding file in x_training.
Moved 1918_nwi_ccap_filled_utm4.tif to /content/moved_y_training because it does not have a corresponding file in x_training.
Moved 764_nwi_ccap_filled_utm4.tif to /content/moved_y_training because it does not have a corresponding file in x_trainin

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/moved_x_training_1year'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')


The number of .tif files in the folder is: 18


pause


In [ ]:
def copy_files_to_new_folder(source_folders, dest_folder):
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    for folder in source_folders:
        # List all files in the current source folder
        files = os.listdir(folder)

        for file in files:
            # Construct the path to the source file
            source_file = os.path.join(folder, file)

            # Check if it's a file (and not a directory)
            if os.path.isfile(source_file):
                # Construct the path to the destination file
                dest_file = os.path.join(dest_folder, file)

                # Copy the file to the destination folder
                shutil.copy(source_file, dest_file)
                print(f"Copied {file} to {dest_folder}")

# Specify the paths to your source folders
source_folders = [
    "/content/ThreeYears/utm4/labels/content/folder_stack/ThreeYears"
]

# Specify the path to your new destination folder
dest_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/lumahai_threeYear_renamed"

# Copy files from both source folders to the destination folder
copy_files_to_new_folder(source_folders, dest_folder)

In [ ]:
def rename_files(folder_path):
    # List all .tif files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]

    for filename in files:
        # Check if the file is already in the desired format
        if "_planet_stack_utm4.tif" in filename:
            continue  # Skip renaming if it's already in the correct format

        # Construct the new filename by replacing '_stacked.tif' with '_planet_stack_utm4.tif'
        new_filename = filename.replace('_stacked.tif', '_planet_stack_utm4.tif')

        # Construct the full old and new file paths
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, new_filename)

        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed '{filename}' to '{new_filename}'")

# Specify the path to your folder
folder_path = "/content/ThreeYears/utm4/labels/content/folder_stack/ThreeYears"

# Call the function for the folder
rename_files(folder_path)

In [ ]:
# Define the source and destination directories
source_dir = "/content/drive/MyDrive/DeepLearning_Project/planet_OneYear/labels"
destination_dir = "/content/labels"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.copy(source_file, destination_file)


In [ ]:
!unzip /content/drive/MyDrive/data/model_template/x_training/x_training_1year/x_training_1year.zip -d /content/OneYear/utm4/labels/

In [ ]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm4/labels/content/exports/nwi_ccap_filled"
destination_dir = "/content/OneYear/utm4/labels"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

# TwoYears UTM 4 Data

In [ ]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_TwoYears_stacked_01.zip -d /content/TwoYears/utm4/images/

In [ ]:
# Define the source and destination directories
source_dir = "/content/TwoYears/utm4/images/content/folder_stack/TwoYears"
destination_dir = "/content/TwoYears/utm4/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2605


# OneYear UTM 5 Data

In [ ]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_OneYear_stacked.zip -d /content/OneYear/utm5/images/

In [ ]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm5/images/content/stack/OneYear"
destination_dir = "/content/OneYear/utm5/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm5/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


In [ ]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_TwoYears_stacked.zip -d /content/TwoYears/utm5/images/

In [ ]:
# Define the source and destination directories
source_dir = "/content/TwoYears/utm5/images/content/stack/TwoYears"
destination_dir = "/content/TwoYears/utm5/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm5/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


# For UTM 4 data, find the unmatched files

In [ ]:
# Specify the folder path
folder_path = '/content/TwoYears/utm5/images'

# Function to extract the numeric part of the filename
def extract_numeric_part(filename):
    match = re.match(r"(\d+)", filename)
    return int(match.group()) if match else None

# List all files in the folder
files = os.listdir(folder_path)

# Extract numeric parts and sort
numeric_parts = [extract_numeric_part(file) for file in files if extract_numeric_part(file) is not None]
sorted_numeric_parts = sorted(numeric_parts)

# Find missing numbers
max_num = sorted_numeric_parts[-1]  # Assuming the largest number in the sequence is the last one
complete_range = set(range(1, max_num + 1))  # Start from 1 to max_num, inclusive
missing_numbers = complete_range - set(sorted_numeric_parts)

# Print missing numbers
print("Missing numbers:", sorted(list(missing_numbers)))

Missing numbers: []


In [ ]:
# from collections import Counter

# # Are there any duplicate files?
# # No duplicate files found.

# # Specify the folder path
# folder_path = '/content/TwoYears/utm5/images'

# # Function to extract the numeric part of the filename
# def extract_numeric_part(filename):
#     match = re.match(r"(\d+)", filename)
#     return int(match.group()) if match else None

# # List all files in the folder
# files = os.listdir(folder_path)

# # Extract numeric parts
# numeric_parts = [extract_numeric_part(file) for file in files if extract_numeric_part(file) is not None]

# # Count occurrences of each numeric part
# numeric_parts_counts = Counter(numeric_parts)

# # Identify and print duplicates
# duplicates = {part: count for part, count in numeric_parts_counts.items() if count > 1}
# if duplicates:
#     print("Duplicate files (numeric part: count):")
#     for part, count in duplicates.items():
#         print(f"{part}: {count} times")
# else:
#     print("No duplicate files found.")

No duplicate files found.


In [ ]:
!mkdir /content/OneYear/utm4/REMOVED

In [ ]:

# Define the source and destination directories
source_folder = "/content/OneYear/utm4/images/"
destination_folder = "/content/OneYear/utm4/REMOVED/"

# List of specific file names to move
files_to_move = ["596_stacked.tif", "597_stacked.tif", "598_stacked.tif", "599_stacked.tif", "600_stacked.tif",
                 "601_stacked.tif", "602_stacked.tif", "603_stacked.tif", "604_stacked.tif", "605_stacked.tif",
                 "606_stacked.tif", "607_stacked.tif", "608_stacked.tif", "609_stacked.tif", "610_stacked.tif",
                 "611_stacked.tif", "612_stacked.tif", "613_stacked.tif", "614_stacked.tif", "1832_stacked.tif",
                 "1833_stacked.tif", "1834_stacked.tif", "1835_stacked.tif", "1836_stacked.tif"]

# Move each file to the destination folder
for filename in files_to_move:
    # Construct the source and destination file paths
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_folder, filename)

    # Check if the source file exists
    if os.path.exists(source_file):
        # Move the file to the destination folder
        shutil.move(source_file, destination_file)
        print(f"File '{filename}' moved successfully.")
    else:
        print(f"File '{filename}' does not exist in the source folder.")


File '596_stacked.tif' does not exist in the source folder.
File '597_stacked.tif' does not exist in the source folder.
File '598_stacked.tif' does not exist in the source folder.
File '599_stacked.tif' does not exist in the source folder.
File '600_stacked.tif' does not exist in the source folder.
File '601_stacked.tif' does not exist in the source folder.
File '602_stacked.tif' does not exist in the source folder.
File '603_stacked.tif' does not exist in the source folder.
File '604_stacked.tif' does not exist in the source folder.
File '605_stacked.tif' does not exist in the source folder.
File '606_stacked.tif' does not exist in the source folder.
File '607_stacked.tif' does not exist in the source folder.
File '608_stacked.tif' does not exist in the source folder.
File '609_stacked.tif' does not exist in the source folder.
File '610_stacked.tif' does not exist in the source folder.
File '611_stacked.tif' does not exist in the source folder.
File '612_stacked.tif' does not exist in

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2605


In [ ]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2605


# Rename files to include _utm4 or _utm5 for iamgery and save to google drive

In [ ]:
def rename_files(folder_path):
    # List all .tif files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]

    for filename in files:
        # Check if the file is already in the desired format
        if "_planet_stack_utm4.tif" in filename:
            continue  # Skip renaming if it's already in the correct format

        # Construct the new filename by replacing '_stacked.tif' with '_planet_stack_utm4.tif'
        new_filename = filename.replace('_stacked.tif', '_planet_stack_utm4.tif')

        # Construct the full old and new file paths
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, new_filename)

        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed '{filename}' to '{new_filename}'")

# Specify the path to your folder
folder_path = "/content/TwoYears/utm4/images"

# Call the function for the folder
rename_files(folder_path)

Renamed '1096_stacked.tif' to '1096_planet_stack_utm4.tif'
Renamed '2064_stacked.tif' to '2064_planet_stack_utm4.tif'
Renamed '343_stacked.tif' to '343_planet_stack_utm4.tif'
Renamed '244_stacked.tif' to '244_planet_stack_utm4.tif'
Renamed '2061_stacked.tif' to '2061_planet_stack_utm4.tif'
Renamed '431_stacked.tif' to '431_planet_stack_utm4.tif'
Renamed '325_stacked.tif' to '325_planet_stack_utm4.tif'
Renamed '2202_stacked.tif' to '2202_planet_stack_utm4.tif'
Renamed '563_stacked.tif' to '563_planet_stack_utm4.tif'
Renamed '1284_stacked.tif' to '1284_planet_stack_utm4.tif'
Renamed '678_stacked.tif' to '678_planet_stack_utm4.tif'
Renamed '2307_stacked.tif' to '2307_planet_stack_utm4.tif'
Renamed '1285_stacked.tif' to '1285_planet_stack_utm4.tif'
Renamed '2217_stacked.tif' to '2217_planet_stack_utm4.tif'
Renamed '1295_stacked.tif' to '1295_planet_stack_utm4.tif'
Renamed '2186_stacked.tif' to '2186_planet_stack_utm4.tif'
Renamed '2149_stacked.tif' to '2149_planet_stack_utm4.tif'
Renamed '

In [ ]:
def copy_files_to_new_folder(source_folders, dest_folder):
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    for folder in source_folders:
        # List all files in the current source folder
        files = os.listdir(folder)

        for file in files:
            # Construct the path to the source file
            source_file = os.path.join(folder, file)

            # Check if it's a file (and not a directory)
            if os.path.isfile(source_file):
                # Construct the path to the destination file
                dest_file = os.path.join(dest_folder, file)

                # Copy the file to the destination folder
                shutil.copy(source_file, dest_file)
                print(f"Copied {file} to {dest_folder}")

# Specify the paths to your source folders
source_folders = [
    "/content/TwoYears/utm4/images"
]

# Specify the path to your new destination folder
dest_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data"

# Copy files from both source folders to the destination folder
copy_files_to_new_folder(source_folders, dest_folder)

Copied 92_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 334_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1324_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1776_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 560_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1505_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 2094_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1123_planet_stack_utm4.tif to /content/drive/

In [ ]:
# Define the folder containing .tif files
images_folder = '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/TwoYear_stack_data'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


# investigate the files across one, two, three year stack for utm 4 and utm 5.

In [ ]:
# Do the number of files in the folders dont match. Look at the data across one, two, three stack folders (UTM5)

def find_matching_files(images_folder, labels_folder):
    # List all .tif files in both folders
    images_files = [f for f in os.listdir(images_folder) if f.endswith('.tif')]
    labels_files = [f for f in os.listdir(labels_folder) if f.endswith('.tif')]

    # Extract the common identifiers from the filenames
    images_ids = set(sorted([f.split('_planet_stack_utm4.tif')[0] for f in images_files]))
    labels_ids = set(sorted([f.split('_planet_stack_utm4.tif')[0] for f in labels_files]))

    # Find the intersection of identifiers, i.e., those that exist in both folders
    matching_ids = images_ids.intersection(labels_ids)

    # Construct the filenames based on matching identifiers
    matching_images = [f"{id}_planet_stack_utm4.tif" for id in matching_ids]
    matching_labels = [f"{id}_planet_stack_utm4.tif" for id in matching_ids]

    return matching_images, matching_labels

# Specify the paths to your folders
images_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data"
labels_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ThreeYear_stack_data"

# Find matching files
matching_images, matching_labels = find_matching_files(images_folder, labels_folder)

# Print the count of matching files
print(f"Found {len(matching_images)} matching image(s) and {len(matching_labels)} matching image(s).")

# Example output: matching filenames
for img, lbl in zip(matching_images, matching_labels):
    print(f"Image: {img} <--> Image: {lbl}")

Found 2605 matching image(s) and 2605 matching image(s).
Image: 1044_planet_stack_utm4.tif <--> Image: 1044_planet_stack_utm4.tif
Image: 2639_planet_stack_utm4.tif <--> Image: 2639_planet_stack_utm4.tif
Image: 753_planet_stack_utm4.tif <--> Image: 753_planet_stack_utm4.tif
Image: 1828_planet_stack_utm4.tif <--> Image: 1828_planet_stack_utm4.tif
Image: 1199_planet_stack_utm4.tif <--> Image: 1199_planet_stack_utm4.tif
Image: 1329_planet_stack_utm4.tif <--> Image: 1329_planet_stack_utm4.tif
Image: 1880_planet_stack_utm4.tif <--> Image: 1880_planet_stack_utm4.tif
Image: 211_planet_stack_utm4.tif <--> Image: 211_planet_stack_utm4.tif
Image: 1187_planet_stack_utm4.tif <--> Image: 1187_planet_stack_utm4.tif
Image: 1995_planet_stack_utm4.tif <--> Image: 1995_planet_stack_utm4.tif
Image: 2430_planet_stack_utm4.tif <--> Image: 2430_planet_stack_utm4.tif
Image: 824_planet_stack_utm4.tif <--> Image: 824_planet_stack_utm4.tif
Image: 1082_planet_stack_utm4.tif <--> Image: 1082_planet_stack_utm4.tif


In [ ]:
import os

# Define the folder paths
folder1_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/OneYear_stack_data"
folder2_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ThreeYear_stack_data"
folder3_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/nwi_ccap_filled"

# # Function to get sorted list of files in a folder
# def get_sorted_files(folder_path):
#     files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
#     sorted_files = sorted(files, key=lambda x: int(x.split('.')[0]))
#     return sorted_files




# Function to get sorted list of files in a folder
def get_sorted_files(folder_path):
    files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    sorted_files = sorted(files, key=lambda x: int(x.split('_')[0]) if x.split('_')[0].isdigit() else x)
    return sorted_files


# Get sorted lists of files in all three folders
folder1_files = get_sorted_files(folder1_path)
folder2_files = get_sorted_files(folder2_path)
folder3_files = get_sorted_files(folder3_path)

# Check if the lists match across all three folders
if folder1_files == folder2_files == folder3_files:
    print("The lists of files in all three folders match.")
else:
    print("The lists of files in all three folders do not match.")

# # Print files in each folder
# print("Files in Folder 1:")
# for file in folder1_files:
#     print(file)

# print("\nFiles in Folder 2:")
# for file in folder2_files:
#     print(file)

print("\nFiles in Folder 3:")
for file in folder3_files:
    print(file)


The lists of files in all three folders do not match.

Files in Folder 3:
1_nwi_ccap_filled_utm4.tif
2_nwi_ccap_filled_utm4.tif
3_nwi_ccap_filled_utm4.tif
4_nwi_ccap_filled_utm4.tif
5_nwi_ccap_filled_utm4.tif
6_nwi_ccap_filled_utm4.tif
7_nwi_ccap_filled_utm4.tif
8_nwi_ccap_filled_utm4.tif
9_nwi_ccap_filled_utm4.tif
10_nwi_ccap_filled_utm4.tif
11_nwi_ccap_filled_utm4.tif
12_nwi_ccap_filled_utm4.tif
13_nwi_ccap_filled_utm4.tif
14_nwi_ccap_filled_utm4.tif
15_nwi_ccap_filled_utm4.tif
16_nwi_ccap_filled_utm4.tif
17_nwi_ccap_filled_utm4.tif
18_nwi_ccap_filled_utm4.tif
19_nwi_ccap_filled_utm4.tif
20_nwi_ccap_filled_utm4.tif
21_nwi_ccap_filled_utm4.tif
22_nwi_ccap_filled_utm4.tif
23_nwi_ccap_filled_utm4.tif
24_nwi_ccap_filled_utm4.tif
25_nwi_ccap_filled_utm4.tif
26_nwi_ccap_filled_utm4.tif
27_nwi_ccap_filled_utm4.tif
28_nwi_ccap_filled_utm4.tif
29_nwi_ccap_filled_utm4.tif
30_nwi_ccap_filled_utm4.tif
31_nwi_ccap_filled_utm4.tif
32_nwi_ccap_filled_utm4.tif
33_nwi_ccap_filled_utm4.tif
34_nwi_ccap

In [ ]:
# prompt: make a sorted list of files in /content/labels and print how many files there are

import os

# Get the list of files in the directory
files = os.listdir("/content/labels")

# Sort the files
files.sort()

# Print the number of files
print(f"Number of files: {len(files)}")

# # Print the sorted list of files
# for filename in files:
#     print(filename)


Number of files: 2374


In [ ]:
# !ls {folder1_path}
# !ls {folder2_path}
!ls {"/content/labels"}

ls: cannot access 'length/content/labels': No such file or directory


In [ ]:
# Define the folder containing .tif files
images_folder = '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2825


# Time to MOVE utm4 and utm5 oneyear, twoyear, threeyear data to folders for modeling

In [ ]:
tif_source  = "/content/ThreeYears/images/x_training_3year/"
export_path = "/content/drive/MyDrive/data/model_template/x_training/x_training_3year/"

# Ensure the destination folder exists
if not os.path.exists(export_path):
    os.makedirs(export_path)

# List all .tif files in the source directory
tif_files = [f for f in os.listdir(tif_source) if f.endswith('.tif')]

# Copy each .tif file to the Google Drive destination folder
for tif_file in tif_files:
    source_path = os.path.join(tif_source, tif_file)
    destination_path = os.path.join(export_path, tif_file)
    shutil.copy(source_path, destination_path)
    print(f"Copied {tif_file} to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.")

Copied 203_planet_stack_utm5.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 1663_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 1971_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 1991_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 307_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 494_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 1783_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 489_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 2367_planet_stack_utm4.tif to /content/drive/MyDrive/data/model_template/x_training/x_training_3year.
Copied 2228_planet_stac

In [ ]:
images_folder = '/content/drive/MyDrive/data/model_template/y_training'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'The number of .tif files in the folder is: {num_tif_files}')


images_folder = '/content/drive/MyDrive/data/model_template/x_training/x_training_1year'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'The number of .tif files in the folder is: {num_tif_files}')


images_folder = '/content/drive/MyDrive/data/model_template/x_training/x_training_2year'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'The number of .tif files in the folder is: {num_tif_files}')


images_folder = '/content/drive/MyDrive/data/model_template/x_training/x_training_3year'
files = os.listdir(images_folder)
tif_files = [file for file in files if file.endswith('.tif')]
num_tif_files = len(tif_files)
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2374
The number of .tif files in the folder is: 2374
The number of .tif files in the folder is: 2374
The number of .tif files in the folder is: 2374
